In [ ]:
root_dir = "output"
method_name = "standard"
benchmark = "tabmwp"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [ ]:
from agential.prompting.standard.prompting import Standard
method = Standard(
        llm=llm,
        benchmark=benchmark,
)

In [ ]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        api_key = open_api,
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    api_key = open_api,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [ ]:
from datasets import load_dataset


data = load_dataset("Arietem/tabmwp")

In [ ]:
from agential.eval.metrics.classification import f1, fuzzy_EM, llm_as_judge_eval, precision, recall, EM
import re

# Initialize score lists
em_scores = []

# Iterate through the first 10 instances in the training data
for index in range(min(len(data['train']['question']), 10)):
    # Extract data for the current instance
    question = data['train']['question'][index]
    table_title = data['train']['table_title'][index]
    table = data['train']['table'][index]
    answer = data['train']["answer"][index]

    # Clean up the answer
    answer = str(answer).replace(',', '')  # Remove commas, convert to string

    # Combine question components
    question = '\n'.join(filter(None, [question, table_title, table]))

    # Inference
    out = method.generate(
        question=question,
        key=answer,
        num_retries=num_retries,
        warming=warming
    )

    # Process the output
    code_str = out.answer.replace("```python", "").replace("```", "").strip()
    pred_answers, _ = safe_execute(code_string=code_str)
    pred_answer = str(pred_answers[0]).lower()

    # Determine the final predicted answer
    if any(word in pred_answer for word in ["yes", "true"]):
        pred_answer = "yes"
    elif any(word in pred_answer for word in ["no", "false"]):
        pred_answer = "no"

    print("Predicted answer:", pred_answer)
    print("Actual answer:", answer)

    # Evaluate correctness
    is_correct = int(EM(pred_answer, answer, is_numeric=True))
    em_scores.append(is_correct)
